In [1]:
import cv2
import os
import re

# Konfigurasi
image_folder = 'D:\\TA_Lalu_Lintas\\TA-Lalu-Lintas\\dataset-mentah\\images'  # Path ke folder gambar
output_yolo_images = 'dataset_yolo/images/'
output_yolo_labels = 'dataset_yolo/labels/'
output_cnn_worn = 'dataset_cnn/helmet_worn/'
output_cnn_not_worn = 'dataset_cnn/helmet_not_worn/'
os.makedirs(output_yolo_images, exist_ok=True)
os.makedirs(output_yolo_labels, exist_ok=True)
os.makedirs(output_cnn_worn, exist_ok=True)
os.makedirs(output_cnn_not_worn, exist_ok=True)

# Variabel Global
drawing = False
ix, iy = -1, -1
boxes = []  # Menyimpan semua bounding box per gambar
current_class = 0  # 0 = Motor, 1 = Helmet

# Fungsi menggambar bounding box
def draw_bbox(event, x, y, flags, param):
    global ix, iy, drawing, img, boxes, current_class

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            img_copy = img.copy()
            color = (0, 255, 0) if current_class == 0 else (255, 0, 0)
            cv2.rectangle(img_copy, (ix, iy), (x, y), color, 2)
            cv2.imshow('Image', img_copy)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        color = (0, 255, 0) if current_class == 0 else (255, 0, 0)
        cv2.rectangle(img, (ix, iy), (x, y), color, 2)
        x_min, y_min = min(ix, x), min(iy, y)
        x_max, y_max = max(ix, y), max(iy, y)
        boxes.append((current_class, x_min, y_min, x_max, y_max))
        cv2.imshow('Image', img)

# Fungsi untuk mendapatkan nomor urut terakhir dari file dalam sebuah folder
def get_last_file_number(folder_path, prefix, extension):
    files = os.listdir(folder_path)
    pattern = re.compile(f'{prefix}_(\\d+){extension}')
    numbers = [int(pattern.match(file).group(1)) for file in files if pattern.match(file)]
    return max(numbers) if numbers else 0

# Fungsi utama
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

for image_file in image_files:
    img_path = os.path.join(image_folder, image_file)
    img = cv2.imread(img_path)
    img_copy = img.copy()
    boxes = []

    cv2.namedWindow('Image')
    cv2.setMouseCallback('Image', draw_bbox)

    while True:
        cv2.imshow('Image', img)
        key = cv2.waitKey(1)

        if key == ord('m'):  # Motor
            current_class = 0

        elif key == ord('h'):  # Helmet
            current_class = 1

        elif key == ord('r'):  # Reset label
            img = img_copy.copy()
            boxes.clear()

        elif key == 13:  # Simpan data jika ada label (Enter)
            if not boxes:
                print(f"Tidak ada label pada gambar {image_file}, tidak disimpan!")
                break

            # Simpan label dalam format YOLO
            last_label_number = get_last_file_number(output_yolo_labels, "frame", ".txt")
            label_file = os.path.join(output_yolo_labels, f'frame_{last_label_number + 1:04d}.txt')

            with open(label_file, 'w') as f:
                motor_bboxes = []
                helmet_bboxes = []
                for class_id, x_min, y_min, x_max, y_max in boxes:
                    x_center = (x_min + x_max) / 2 / img.shape[1]
                    y_center = (y_min + y_max) / 2 / img.shape[0]
                    width = (x_max - x_min) / img.shape[1]
                    height = (y_max - y_min) / img.shape[0]
                    f.write(f'{class_id} {x_center} {y_center} {width} {height}\n')

                    if class_id == 0:
                        motor_bboxes.append((x_min, y_min, x_max, y_max))
                    elif class_id == 1:
                        helmet_bboxes.append((x_min, y_min, x_max, y_max))

            # Simpan gambar YOLO
            last_image_number = get_last_file_number(output_yolo_images, "frame", ".jpg")
            image_file_output = os.path.join(output_yolo_images, f'frame_{last_image_number + 1:04d}.jpg')
            cv2.imwrite(image_file_output, img)

            # Ekstrak kepala untuk dataset CNN
            for motor_bbox in motor_bboxes:
                x_min, y_min, x_max, y_max = motor_bbox
                head_crop = img[y_min:y_min + int(0.3 * (y_max - y_min)), x_min:x_max]

                helmet_found = False
                for helmet_bbox in helmet_bboxes:
                    hx_min, hy_min, hx_max, hy_max = helmet_bbox
                    if hx_min >= x_min and hy_min >= y_min and hx_max <= x_max and hy_max <= y_max:
                        helmet_found = True
                        break

                if helmet_found:
                    last_worn = get_last_file_number(output_cnn_worn, "head", ".jpg") + 1
                    cv2.imwrite(os.path.join(output_cnn_worn, f'head_{last_worn:04d}.jpg'), head_crop)
                else:
                    last_not_worn = get_last_file_number(output_cnn_not_worn, "head", ".jpg") + 1
                    cv2.imwrite(os.path.join(output_cnn_not_worn, f'head_{last_not_worn:04d}.jpg'), head_crop)

            print(f"Label dan gambar untuk {image_file} telah disimpan.")
            break

        elif key == ord('q'):  # Keluar program
            print("Proses dihentikan oleh pengguna.")
            cv2.destroyAllWindows()
            exit()

    cv2.destroyWindow('Image')


Tidak ada label pada gambar BikesHelmets10.png, tidak disimpan!
Tidak ada label pada gambar BikesHelmets103.png, tidak disimpan!
Tidak ada label pada gambar BikesHelmets108.png, tidak disimpan!
Tidak ada label pada gambar BikesHelmets109.png, tidak disimpan!
Tidak ada label pada gambar BikesHelmets111.png, tidak disimpan!
Tidak ada label pada gambar BikesHelmets112.png, tidak disimpan!
Tidak ada label pada gambar BikesHelmets113.png, tidak disimpan!
Tidak ada label pada gambar BikesHelmets119.png, tidak disimpan!
Tidak ada label pada gambar BikesHelmets120.png, tidak disimpan!
Tidak ada label pada gambar BikesHelmets121.png, tidak disimpan!
Tidak ada label pada gambar BikesHelmets125.png, tidak disimpan!
Tidak ada label pada gambar BikesHelmets13.png, tidak disimpan!
Tidak ada label pada gambar BikesHelmets133.png, tidak disimpan!
Tidak ada label pada gambar BikesHelmets135.png, tidak disimpan!
Tidak ada label pada gambar BikesHelmets139.png, tidak disimpan!
Tidak ada label pada gambar